In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np

# Load the dataset
dataset = pd.read_csv('Crop_recommendation.csv')

# Inspect the first few rows
print(dataset.head())

# Features and labels
X = dataset[['temperature', 'humidity', 'ph', 'water availability', 'season']]
y = dataset['label']

# Encode the categorical 'season' and 'label' columns
label_encoder = LabelEncoder()
X['season'] = label_encoder.fit_transform(X['season'])
y = label_encoder.fit_transform(y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature Scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

   temperature   humidity        ph  water availability season label
0    20.879744  82.002744  6.502985          202.935536  rainy  rice
1    21.770462  80.319644  7.038096          226.655537  rainy  rice
2    23.004459  82.320763  7.840207          263.964248  rainy  rice
3    26.491096  80.158363  6.980401          242.864034  rainy  rice
4    20.130175  81.604873  7.628473          262.717340  rainy  rice


<ipython-input-2-d1c3f75bcb44>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['season'] = label_encoder.fit_transform(X['season'])


In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Check if GPU is available
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
else:
    print(f'Found GPU at: {device_name}')

# Build a Sequential Model
model = Sequential()

# Input layer
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.3))

# Hidden layers
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))

# Output layer
model.add(Dense(len(np.unique(y)), activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=2)

# Evaluate the model on the test data
score = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {score[1] * 100:.2f}%")

Found GPU at: /device:GPU:0


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
35/35 - 6s - 158ms/step - accuracy: 0.1705 - loss: 2.4185 - val_accuracy: 0.4321 - val_loss: 2.1843
Epoch 2/50
35/35 - 0s - 3ms/step - accuracy: 0.3696 - loss: 2.0014 - val_accuracy: 0.6179 - val_loss: 1.5918
Epoch 3/50
35/35 - 0s - 4ms/step - accuracy: 0.4875 - loss: 1.5607 - val_accuracy: 0.7536 - val_loss: 1.0614
Epoch 4/50
35/35 - 0s - 4ms/step - accuracy: 0.6036 - loss: 1.2044 - val_accuracy: 0.7929 - val_loss: 0.7561
Epoch 5/50
35/35 - 0s - 3ms/step - accuracy: 0.6554 - loss: 1.0084 - val_accuracy: 0.8821 - val_loss: 0.5464
Epoch 6/50
35/35 - 0s - 3ms/step - accuracy: 0.6768 - loss: 0.8764 - val_accuracy: 0.9071 - val_loss: 0.4366
Epoch 7/50
35/35 - 0s - 3ms/step - accuracy: 0.7188 - loss: 0.7650 - val_accuracy: 0.9107 - val_loss: 0.3719
Epoch 8/50
35/35 - 0s - 3ms/step - accuracy: 0.7402 - loss: 0.7013 - val_accuracy: 0.9179 - val_loss: 0.3213
Epoch 9/50
35/35 - 0s - 3ms/step - accuracy: 0.7768 - loss: 0.5943 - val_accuracy: 0.9107 - val_loss: 0.2851
Epoch 10/50
35/35

In [4]:
# Save the trained model as an .h5 file
model.save('/content/crop_recommendation_model.h5')

In [5]:
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('crop_recommendation_model.h5')